## 🦀 Topic Partitions

![Partitions](images/partitions.png)

Partitions are a way to distribute our dataset, Apache Kafka guarantees the correct ordering only in events registered in the same partitions. 

Let's create a new producer

In [ ]:
from kafka import KafkaProducer
import json
from config.kafka_config import *

producer = KafkaProducer(
        bootstrap_servers=hostname+":"+str(port),
        security_protocol="SSL",
        ssl_cafile=cert_folder+"/ca.pem",
        ssl_certfile=cert_folder+"/service.cert",
        ssl_keyfile=cert_folder+"/service.key",
        value_serializer=lambda v: json.dumps(v).encode('ascii'),
        key_serializer=lambda v: json.dumps(v).encode('ascii')   
    )

Create a topic with **two partitions**, check `num_partitions=2`

In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic


admin = KafkaAdminClient(
        client_id ='admin',
        bootstrap_servers=hostname+":"+str(port),
        security_protocol="SSL",
        ssl_cafile=cert_folder+"/ca.pem",
        ssl_certfile=cert_folder+"/service.cert",
        ssl_keyfile=cert_folder+"/service.key",
    )

topic_name_partitioned=topic_name+"_partitioned"

topic=NewTopic(name=topic_name_partitioned, num_partitions=2, replication_factor=1)

admin.create_topics([topic], timeout_ms=int(timeout_ms))

And now push data to the two partitions

In [ ]:
producer.send(topic_name_partitioned,
              key={"id":1},
              value={"name":"👨 Frank", "pizza":"Margherita 🍕"},
              partition=0
             )
producer.send(topic_name_partitioned,
              key={"id":2},
              value={"name":"👩 Adele", "pizza":"Hawaii 🍕+🍍+🥓"},
              partition=1
             )
producer.flush()

Insert more data

In [ ]:
producer.send(topic_name_partitioned,
              key={"id":1},
              value={"name":"🙎 Mark", "pizza":"Banana 🍕+🍌"},
              partition=0
             )
producer.send(topic_name_partitioned,
              key={"id":2},
              value={"name":"👨 Jan", "pizza":"Mushrooms 🍕+🍄"},
              partition=1
             )
producer.flush()